<a href="https://colab.research.google.com/github/Iroda732/fork1/blob/main/Pneumonia_X_rays.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# IMPORTANT: SOME KAGGLE DATA SOURCES ARE PRIVATE
# RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES.
import kagglehub
kagglehub.login()


In [ ]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

pnevmoniya_path = kagglehub.competition_download('pnevmoniya')

print('Data source import complete.')


In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
!pip install tensorflow numpy matplotlib pandas opencv-python scikit-learn

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import fastai
import torch
from fastai.vision.all import *

In [ ]:
print(fastai.__version__)

In [ ]:
path = Path('/kaggle/input/pnevmoniya/train')

In [ ]:
train_datagen = ImageDataGenerator(rescale=1./255,
                                   rotation_range=20,
                                   zoom_range=0.2,
                                   shear_range=0.2,
                                   horizontal_flip=True,
                                   validation_split=0.2)

In [ ]:
train_generator = train_datagen.flow_from_directory(
    path,
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary',
    subset='training')

In [ ]:
validation_generator = train_datagen.flow_from_directory(
    path,
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary',
    subset='validation')


In [ ]:
sample_images, sample_labels = next(train_generator)
plt.figure(figsize=(10,10))
for i in range(9):
    plt.subplot(3,3,i+1)
    plt.imshow(sample_images[i])
    plt.axis('off')
plt.show()

In [ ]:
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam


In [ ]:
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

In [ ]:
for layer in base_model.layers:
    layer.trainable = False

In [ ]:
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(512, activation='relu')(x)
x = Dense(128, activation='relu')(x)
predictions = Dense(1, activation='sigmoid')(x)  # Sigmoid for binary classification

In [ ]:
model = Model(inputs=base_model.input, outputs=predictions)

In [ ]:
model.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
model.summary()

In [ ]:
history = model.fit(
    train_generator,
    epochs=10,
    validation_data=validation_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    validation_steps=validation_generator.samples // validation_generator.batch_size)

In [ ]:
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.legend()
plt.show()

In [ ]:
for layer in base_model.layers[-30:]:
    layer.trainable = True

In [ ]:
model.compile(optimizer=Adam(learning_rate=1e-5), loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
fine_tune_history = model.fit(
    train_generator,
    epochs=5,
    validation_data=validation_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    validation_steps=validation_generator.samples // validation_generator.batch_size)

In [ ]:
plt.plot(fine_tune_history.history['accuracy'], label='Fine-tune Train Accuracy')
plt.plot(fine_tune_history.history['val_accuracy'], label='Fine-tune Validation Accuracy')
plt.legend()
plt.show()

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix

In [ ]:
validation_generator.reset()
predictions = model.predict(validation_generator, steps=validation_generator.samples // validation_generator.batch_size)
y_pred = np.where(predictions > 0.5, 1, 0)
y_true = validation_generator.classes[:len(y_pred)]

In [ ]:
print(classification_report(y_true, y_pred, target_names=['Normal', 'Pneumonia']))
print(confusion_matrix(y_true, y_pred))

In [ ]:
model.save('pneumonia_detection_model.h5')

In [ ]:
from tensorflow.keras.preprocessing import image

img_path = '/kaggle/input/pnevmoniya/test/test_img_10001.jpeg'
img = image.load_img(img_path, target_size=(224, 224))
img_array = image.img_to_array(img) / 255.0
img_array = np.expand_dims(img_array, axis=0)

In [ ]:
import tensorflow as tf

In [ ]:
model = tf.keras.models.load_model('pneumonia_detection_model.h5')
prediction = model.predict(img_array)
print(f'Prediction: {"Pneumonia" if prediction > 0.5 else "Normal"}')

In [ ]:
test_images = ['/kaggle/input/pnevmoniya/test/test_img_10001.jpeg', '/kaggle/input/pnevmoniya/test/test_img_10002.jpeg', '/kaggle/input/pnevmoniya/test/test_img_10003.jpeg', '/kaggle/input/pnevmoniya/test/test_img_10004.jpeg', '/kaggle/input/pnevmoniya/test/test_img_10005.jpeg']
predictions = []


In [ ]:
for img_path in test_images:
    img = image.load_img(img_path, target_size=(224, 224))
    img_array = image.img_to_array(img) / 255.0
    img_array = np.expand_dims(img_array, axis=0)
    pred = model.predict(img_array)
    predictions.append(pred[0][0])

In [ ]:
submission = pd.DataFrame({
    'id': [os.path.basename(img_path) for img_path in test_images],
    'label': predictions
})

In [ ]:
submission.to_csv('submission.csv', index=False)